# geo-fencing

In [1]:
import arcgis
from arcgis.gis import GIS
import arcgis.geocoding as geocoding
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import osmapi
import csv
import pandas as pd

In [2]:
my_gis = GIS()

In [3]:
hello_map = my_gis.map("Санкт-Петербург", zoomlevel=15)

In [4]:
hello_map.basemap = 'osm'

In [5]:
class Adres_read():
    def __init__(self, adress):
        self.__geolocator = Nominatim(user_agent="specify_your_app_name_here")
        self.__location = self.__geolocator.geocode(adress)
        # self.__location_place = (self.__location.latitude, self.__location.longitude)

        self.__bounding = self.__location.raw['boundingbox']

    def get_points(self):
        return self.__bounding

In [6]:
class Osm_reader():
#     def __init__(self, min_lon, min_lat, max_lon, max_lat):
    def __init__(self, lil):
#         print(lil)
        self.__api = osmapi.OsmApi()
#         self.__tmp = self.__api.Map(min_lon, min_lat, max_lon, max_lat)
        self.__tmp = self.__api.Map(float(lil[2]), float(lil[0]), float(lil[3]), float(lil[1]))
        self.__ret = []

        for e in self.__tmp:
            try:
                try:
                    try:
                        type_ = e['type']
                        timestamp = e['data']['timestamp']
                        id = e['data']['id']
                        lat = e['data']['lat']
                        lon = e['data']['lon']
                        uid = e['data']['uid']
                        user = str(e['data']['user'])
                        tag = e['data']['tag']
                        delta_days = (osmapi.datetime.today() - timestamp).days

                        if type_ == 'node':
                            self.__ret.append([id, lat, lon, uid, timestamp, user])

                    except KeyError:
                        pass
                except IndexError:
                    pass
            except UnicodeEncodeError as err:
                print(err, str(e) + '\n')

    def create_file(self):
        with open('dumps.csv', "w", newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file, delimiter=',')
            for line in self.__ret:
#                 print(line)
                writer.writerow(line)
                
            csv_file.close()

    def get_values(self):
        return self.__ret

In [7]:
def find_addr(hello_map, g):
#     try:
    hello_map.draw(g)
    geocoded = geocoding.reverse_geocode(g)
    address = geocoded['address']['Match_addr']
    adres_read = Adres_read(address)
    print(adres_read.get_points())

    try:
        print('creating')
        a = Osm_reader(adres_read.get_points())
        a.create_file()
        print('file created')
    except:
        print("beda")


hello_map.on_click(find_addr)
# hello_map.on_click(add_costil())
hello_map

MapView(layout=Layout(height='400px', width='100%'), zoom=15.0)

['59.9365155', '59.9365886', '30.300875', '30.3009767']
creating
file created


In [8]:
# def add_costil():
kostil = pd.read_csv('dumps.csv')
kostil.columns = ['id', 'lat', 'lon', 'uid', 'timestamp', 'user']
if len(kostil) > 999:
    kostil = kostil[0:1000]
    hello_map.add_layer(my_gis.content.import_data(kostil))
else:
    hello_map.add_layer(my_gis.content.import_data(kostil))

In [71]:
# hello_map.add_layer(my_gis.content.import_data(kostil))

In [80]:
import pandas as pd
import xml.etree.cElementTree as et
import xml


def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None


parsedXML = et.parse('2860167.gpx')
dfcols = ['time', 'lat', 'lon']
df_xml2 = pd.DataFrame(columns=dfcols)

file = xml.dom.minidom.parse('2860167.gpx')
file.normalize()
time = []
a = file.getElementsByTagName('time')
for e in a:
    time.append(e.firstChild.data)
#     print(e.firstChild.data)
x = 0
for node in parsedXML.getroot():
#     time = node.attrib.get('time')
    lat = node.attrib.get('lat')
    lon = node.attrib.get('lon')
    
    df_xml2 = df_xml2.append(
        pd.Series([time[x] ,lat, lon], index=dfcols),
        ignore_index=True)
    x += 1
    
    
df_xml2 = df_xml2[:]
df_xml2.head()
# df_xml.describe()

,time,lat,lon
0,2018-12-08T10:12:04Z,51.94345893144488,15.49198144879156
1,2018-12-08T10:13:19Z,51.94363897126528,15.493174279759414
2,2018-12-08T10:14:34Z,51.94438314560817,15.496343190340847
3,2018-12-08T10:24:14Z,51.94670125293291,15.503452333217876
4,2018-12-08T10:24:59Z,51.94680540152711,15.502982348359335


In [54]:
hello_map.add_layer(my_gis.content.import_data(df_xml2))


In [73]:
time_max = df_xml2.max()
time_max

time    2018-12-08T13:52:46Z
dtype: object

In [74]:
time_min = df_xml2.min()
time_min

time    2018-12-08T10:09:41Z
dtype: object

In [75]:
lat1 = 51.9434589314449
lon1 = 15.4919814487916

In [76]:
lat2 = 51.9515314428726
lon2 = 15.5220052294499

In [117]:
from datetime import datetime
area_check = []
# lat = []
# lon = []
# area_cols = ['lat', 'lon']
# df_xml2['lon'][0]
# len(df_xml2)

df_xml2 = df_xml2[:-1]


for e in range(len(df_xml2)):
    if float(df_xml2['lat'][e]) > lat1 and float(df_xml2['lat'][e]) < lat2 and float(df_xml2['lon'][e]) > lon1 and float(df_xml2['lon'][e]) < lon2:
        area_check.append([df_xml2['time'][e], df_xml2['lat'][e], df_xml2['lon'][e]])


print(area_check)
dates = []
for e in area_check:
    date = datetime.strptime(e[0], '%Y-%m-%dT%H:%M:%SZ')
    dates.append(date)
    print(date)


[['2018-12-08T10:13:19Z', '51.94363897126528', '15.493174279759414'], ['2018-12-08T10:14:34Z', '51.94438314560817', '15.496343190340847'], ['2018-12-08T10:24:14Z', '51.94670125293291', '15.503452333217876'], ['2018-12-08T10:24:59Z', '51.94680540152711', '15.502982348359335'], ['2018-12-08T10:26:24Z', '51.94662435470329', '15.503618772051173']]
2018-12-08 10:13:19
2018-12-08 10:14:34
2018-12-08 10:24:14
2018-12-08 10:24:59
2018-12-08 10:26:24


In [120]:
print(min(dates))

2018-12-08 10:13:19


In [122]:
print(max(dates))

2018-12-08 10:26:24


In [123]:
print("Время нахождения:", max(dates) - min(dates))

Время нахождения: 0:13:05
